In [2]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC, SVR
import seaborn as sns
import os
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
import numpy as np
import pandas as pd
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report, roc_curve, \
RocCurveDisplay, roc_auc_score, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import log_loss
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier

In [3]:
hr = pd.read_csv('Datasets/HR_comma_sep.csv')
hr.dropna()
hr

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.10,0.77,6,247,4,0,1,0,sales,low
3,0.92,0.85,5,259,5,0,1,0,sales,low
4,0.89,1.00,5,224,5,0,1,0,sales,low
...,...,...,...,...,...,...,...,...,...,...
14990,0.40,0.57,2,151,3,0,1,0,support,low
14991,0.37,0.48,2,160,3,0,1,0,support,low
14992,0.37,0.53,2,143,3,0,1,0,support,low
14993,0.11,0.96,6,280,4,0,1,0,support,low


In [4]:
X,y = hr.drop('left',axis=1),hr['left']
ohe = OneHotEncoder(drop='first',sparse_output=False).set_output(transform='pandas')

from sklearn.compose import ColumnTransformer, make_column_selector

col_trnf = ColumnTransformer([('OHE', ohe, make_column_selector(dtype_include=object))],
                              remainder='passthrough', verbose_feature_names_out=False)
col_trnf = col_trnf.set_output(transform='pandas')
X = col_trnf.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=25,stratify=y)

In [5]:
dtc1 = DecisionTreeClassifier(random_state=25, max_depth=3)
dtc2 = DecisionTreeClassifier(random_state=25, max_depth=2)
lr = LogisticRegression()
voting = VotingClassifier(estimators=[('TREE1', dtc1),('TREE2', dtc2), ('LR', lr)])
voting.fit(X_train, y_train)
y_pred = voting.predict(X_test)
print(accuracy_score(y_test, y_pred))


0.902200488997555


c:\Users\Saurav Dani\AppData\Local\Programs\Python\Python314\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [14]:
scaler = StandardScaler()

pipe_knn = Pipeline([
    ('SCL', scaler),
    ('KNN', KNeighborsClassifier(n_neighbors=5))
])

pipe_svm = Pipeline([
    ('SCL', scaler),
    ('SVM', SVC(kernel='rbf', probability=True))
])

dtc1 = DecisionTreeClassifier(random_state=25, max_depth=4)

voting = VotingClassifier(
    estimators=[
        ('TREE1', dtc1),
        ('KNN', pipe_knn),
        ('SVM', pipe_svm)
    ],
    voting='soft'
)

voting.fit(X_train, y_train)

y_pred = voting.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

y_pred_proba = voting.predict_proba(X_test)
print("ROC AUC:", roc_auc_score(y_test, y_pred_proba[:, 1]))

print("LogLoss Score: ", log_loss(y_test, y_pred_proba))

Accuracy: 0.9668815292287175
ROC AUC: 0.9865754709010285
LogLoss Score:  0.1098860716233828
